This is our grid search document! Note that there are some keyboard interrupts in our output, because we ran everything again just to make sure it works prior to our final submission! Because the full search takes a long time, we interrupted the kernel. The code works!

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

In [2]:
# ANN model with 2 hidden layers

print(f"{'~'*15} Building ANN model with 2 hidden layers {'~'*15}\n")

# Load the dataset
df = pd.read_csv("../backend/Datasets/COALINDIA.csv")

# Feature selection and preprocessing
dropped_features = ['Date', 'Symbol', 'Series', 
                    'Trades', 'Turnover', 'Deliverable Volume', 
                    '%Deliverble', 'Last', 'VWAP', 'Prev Close']
df.drop(dropped_features, axis=1, inplace=True)

# Define features (X) and target (Y)
X = df.drop('Close', axis=1)
Y = df['Close']

# Scale the data
scaler_X = MinMaxScaler()
scaler_Y = MinMaxScaler()
X = scaler_X.fit_transform(X.values)
Y = scaler_Y.fit_transform(Y.values.reshape(-1, 1))

# Split the data into training and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42, shuffle=False)

~~~~~~~~~~~~~~~ Building ANN model with 2 hidden layers ~~~~~~~~~~~~~~~



In [3]:
def create_model_A(activation='relu', epochs=100, batch_size=32, op_learning_rate=1e-5, hidden=128, dropout=0.15):
    # Build a neural network model with 2 hidden layers
    # You can experiment with different architectures, including the number of layers and neurons.
    model = Sequential()
    model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(hidden, activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(1))

    # Compile the model
    optimizer = Adam(learning_rate=op_learning_rate)  # Experiment with learning rate
    model.compile(optimizer=optimizer, loss='mean_squared_error')

    # Train the model
    history = model.fit(X_train, Y_train, epochs=100, batch_size=32, validation_data=(X_test, Y_test), verbose=0)

    # Make predictions
    Y_pred = model.predict(X_test)

    # Inverse transform the scaled values
    Y_test_original = scaler_Y.inverse_transform(Y_test)
    Y_pred_original = scaler_Y.inverse_transform(Y_pred)

    print(f"Activation: {activation}\nEpoch: {epochs}\nBatch Size: {batch_size}\nOptimizer learning rate: {op_learning_rate}")
    print(f"Hidden: {hidden}\nDropout: {dropout}")
    # Calculate MSE and R2
    mse = mean_squared_error(Y_test_original, Y_pred_original)
    print(f"Mean Squared Error: {mse}")
    metric = tf.keras.metrics.R2Score()
    metric.update_state(Y_test_original, Y_pred_original)
    r2 = metric.result().numpy()
    print("R^2:", metric.result().numpy())
    print("-"*64)
    return mse, r2

In [4]:
ret = []
for activation in ['relu', 'sigmoid', 'tanh']:
    for epochs in [50, 75, 100, 150]:
        for batch_size in [24, 32, 40]:
            for op_learning_rate in [1e-5, 1e-4, 1e-3]:
                for hidden in [32, 64, 128]:
                    for dropout in [0.05, 0.1, 0.15]:
                        mse, r2 = create_model_A(activation=activation, epochs=epochs, batch_size=batch_size, op_learning_rate=op_learning_rate, hidden=hidden, dropout=dropout)
                        ret.append([activation, epochs, batch_size, op_learning_rate, hidden, dropout, mse, r2])
    

17/17 [==============================] - 0s 510us/step


Activation: relu
Epoch: 50
Batch Size: 24
Optimizer learning rate: 1e-05
Hidden: 32
Dropout: 0.05
Mean Squared Error: 121.92725769397497
R^2: 0.9375689
----------------------------------------------------------------
17/17 [==============================] - 0s 494us/step


Activation: relu
Epoch: 50
Batch Size: 24
Optimizer learning rate: 1e-05
Hidden: 32
Dropout: 0.1
Mean Squared Error: 615.9689242242055
R^2: 0.6846019
----------------------------------------------------------------
17/17 [==============================] - 0s 489us/step


Activation: relu
Epoch: 50
Batch Size: 24
Optimizer learning rate: 1e-05
Hidden: 32
Dropout: 0.15
Mean Squared Error: 451.27214917966967
R^2: 0.7689325
----------------------------------------------------------------


KeyboardInterrupt: 

In [6]:
print("Best MSE for ANN:", sorted(ret, key=lambda x:x[-2])[0])
print("Best R^2 for ANN:", sorted(ret, key=lambda x:-x[-1])[0])

Best MSE for ANN: ['relu', 50, 24, 1e-05, 32, 0.05, 121.92725769397497, 0.9375689]
Best R^2 for ANN: ['relu', 50, 24, 1e-05, 32, 0.05, 121.92725769397497, 0.9375689]


In [7]:
def create_model_P(activation=None, epochs=100, batch_size=32, op_learning_rate=1e-5):
    model = Sequential()
    model.add(Dense(1, input_dim=X_train.shape[1], activation=activation))

    # Compile the model
    optimizer = Adam(learning_rate=op_learning_rate)  # Experiment with learning rate
    model.compile(optimizer=optimizer, loss='mean_squared_error')

    # Train the model
    history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, Y_test), verbose=0)

    # Make predictions
    Y_pred = model.predict(X_test)

    # Inverse transform the scaled values
    Y_test_original = scaler_Y.inverse_transform(Y_test)
    Y_pred_original = scaler_Y.inverse_transform(Y_pred)

    print(f"Activation: {activation}\nEpoch: {epochs}\nBatch Size: {batch_size}\nOptimizer learning rate: {op_learning_rate}")
    # Calculate MSE
    mse = mean_squared_error(Y_test_original, Y_pred_original)
    print(f"Mean Squared Error (Perceptron): {mse}")
    metric = tf.keras.metrics.R2Score()
    metric.update_state(Y_test_original, Y_pred_original)
    r2 = metric.result().numpy()
    print("R^2:", metric.result().numpy())
    print("-"*64)
    return mse, r2


In [8]:
ret = []
for activation in [None, 'relu', 'sigmoid', 'tanh']:
    for epochs in [50, 75, 100, 150]:
        for batch_size in [24, 32, 40]:
            for op_learning_rate in [1e-5, 1e-4, 1e-3]:
                mse, r2 = create_model_P(activation=activation, epochs=epochs, batch_size=batch_size, op_learning_rate=op_learning_rate)
                ret.append([activation, epochs, batch_size, op_learning_rate, mse, r2])
    

17/17 [==============================] - 0s 471us/step


Activation: None
Epoch: 50
Batch Size: 24
Optimizer learning rate: 1e-05
Mean Squared Error (Perceptron): 22709.59449516499
R^2: -10.628126
----------------------------------------------------------------
17/17 [==============================] - 0s 556us/step


Activation: None
Epoch: 50
Batch Size: 24
Optimizer learning rate: 0.0001
Mean Squared Error (Perceptron): 5223.882143812489
R^2: -1.6748145
----------------------------------------------------------------
17/17 [==============================] - 0s 518us/step


Activation: None
Epoch: 50
Batch Size: 24
Optimizer learning rate: 0.001
Mean Squared Error (Perceptron): 21.947583946001345
R^2: 0.988762
----------------------------------------------------------------
17/17 [==============================] - 0s 453us/step


Activation: None
Epoch: 50
Batch Size: 32
Optimizer learning rate: 1e-05
Mean Squared Error (Perceptron): 4861.116664524157
R^2: -1.4890661
----------------------------------------------------------------
17/17 [==============================] - 0s 455us/step


...

Activation: None
Epoch: 50
Batch Size: 32
Optimizer learning rate: 0.001
Mean Squared Error (Perceptron): 548.1260386994243
R^2: 0.71933985
----------------------------------------------------------------
17/17 [==============================] - 0s 451us/step


Activation: None
Epoch: 50
Batch Size: 40
Optimizer learning rate: 1e-05
Mean Squared Error (Perceptron): 13625.950988127473
R^2: -5.9769745
----------------------------------------------------------------
17/17 [==============================] - 0s 471us/step


Activation: None
Epoch: 50
Batch Size: 40
Optimizer learning rate: 0.0001
Mean Squared Error (Perceptron): 2238.9801107860917
R^2: -0.146438
----------------------------------------------------------------
17/17 [==============================] - 0s 460us/step


Activation: None
Epoch: 50
Batch Size: 40
Optimizer learning rate: 0.001
Mean Squared Error (Perceptron): 5779.29742804678
R^2: -1.959207
----------------------------------------------------------------


KeyboardInterrupt: 

In [10]:
print("Best MSE for Perceptron:", sorted(ret, key=lambda x:x[-2])[0])
print("Best R^2 for Perceptron:", sorted(ret, key=lambda x:-x[-1])[0])

Best MSE for Perceptron: [None, 50, 24, 0.001, 21.947583946001345, 0.988762]
Best R^2 for Perceptron: [None, 50, 24, 0.001, 21.947583946001345, 0.988762]


In [11]:
def create_model_D(activation='tanh', epochs=100, batch_size=32, op_learning_rate=1e-5, hidden=128, layers=11, dropout=0.1):
    # Build a neural network model with 2 hidden layers
    # You can experiment with different architectures, including the number of layers and neurons.
    # Building DNN model with 11 hidden layers
    model = Sequential()

    # Input layer
    # Input layer will have same number of neurons as number of feature variables
    model.add(Dense(64, input_dim=X_train.shape[1], activation=activation))

    # Hidden layers
    # Play around with number of neurons in each hidden layer.
    # Too many neurons leads to overcomplexity, not enough means too simple
    # Tanh activation function here is used b/c it is recommended to use
    # when there are more hidden layers.
    for _ in range(layers-1):
        model.add(Dense(hidden, activation=activation))
        model.add(Dropout(dropout)) # This helps with preventing overfitting

    # Output layer
    # Output layer will have 1 neuron b/c there's only 1 target variable
    model.add(Dense(1))

    # Compile the model
    optimizer = Adam(learning_rate=op_learning_rate)  # Experiment with learning rate
    model.compile(optimizer=optimizer, loss='mean_squared_error')

    # Train the model
    history = model.fit(X_train, Y_train, epochs=100, batch_size=32, validation_data=(X_test, Y_test), verbose=0)

    # Make predictions
    Y_pred = model.predict(X_test)

    # Inverse transform the scaled values
    Y_test_original = scaler_Y.inverse_transform(Y_test)
    Y_pred_original = scaler_Y.inverse_transform(Y_pred)

    print(f"Activation: {activation}\nEpoch: {epochs}\nBatch Size: {batch_size}\nOptimizer learning rate: {op_learning_rate}")
    print(f"Hidden: {hidden}\nLayers: {layers}\nDropout: {dropout}")
    # Calculate MSE and R2
    mse = mean_squared_error(Y_test_original, Y_pred_original)
    print(f"Mean Squared Error: {mse}")
    metric = tf.keras.metrics.R2Score()
    metric.update_state(Y_test_original, Y_pred_original)
    r2 = metric.result().numpy()
    print("R^2:", metric.result().numpy())
    print("-"*64)
    return mse, r2

In [12]:
m, r = create_model_D()

17/17 [==============================] - 0s 794us/step
Activation: tanh
Epoch: 100
Batch Size: 32
Optimizer learning rate: 1e-05
Hidden: 128
Layers: 11
Dropout: 0.1
Mean Squared Error: 15.944859824421128
R^2: 0.99183565
----------------------------------------------------------------


In [13]:
# GRID SEARCH

ret = []
for activation in ['relu', 'sigmoid', 'tanh']:
    for epochs in [50, 75, 100, 150]:
        for batch_size in [24, 32, 40]:
            for op_learning_rate in [1e-5, 1e-4, 1e-3]:
                for hidden in [32, 64, 128]:
                    for layers in [6, 9, 11]:
                        for dropout in [0.05, 0.1, 0.15]:
                            mse, r2 = create_model_D(activation=activation, epochs=epochs, batch_size=batch_size, op_learning_rate=op_learning_rate, hidden=hidden, layers=layers, dropout=dropout)
                            ret.append([activation, epochs, batch_size, op_learning_rate, hidden, layers, dropout, mse, r2])
    

17/17 [==============================] - 0s 534us/step


Activation: relu
Epoch: 50
Batch Size: 24
Optimizer learning rate: 1e-05
Hidden: 32
Layers: 6
Dropout: 0.05
Mean Squared Error: 2702.9697519684546
R^2: -0.38401723
----------------------------------------------------------------
17/17 [==============================] - 0s 554us/step


Activation: relu
Epoch: 50
Batch Size: 24
Optimizer learning rate: 1e-05
Hidden: 32
Layers: 6
Dropout: 0.1
Mean Squared Error: 3521.0750428373653
R^2: -0.8029164
----------------------------------------------------------------
17/17 [==============================] - 0s 538us/step


Activation: relu
Epoch: 50
Batch Size: 24
Optimizer learning rate: 1e-05
Hidden: 32
Layers: 6
Dropout: 0.15
Mean Squared Error: 3316.4900278458604
R^2: -0.6981616
----------------------------------------------------------------
17/17 [==============================] - 0s 687us/step


Activation: relu
Epoch: 50
Batch Size: 24
Optimizer learning rate: 1e-05
Hidden: 32
Layers: 9
Dropout: 0.05
Mean Squared Error: 2042.6924079422283
R^2: -0.045931578
----------------------------------------------------------------


KeyboardInterrupt: 

In [14]:
print("Best MSE for DNN:", sorted(ret, key=lambda x:x[-2])[0])
print("Best R^2 for DNN:", sorted(ret, key=lambda x:-x[-1])[0])

Best MSE for DNN: ['relu', 50, 24, 1e-05, 32, 9, 0.05, 2042.6924079422283, -0.045931578]
Best R^2 for DNN: ['relu', 50, 24, 1e-05, 32, 9, 0.05, 2042.6924079422283, -0.045931578]
